<a href="https://colab.research.google.com/github/manargani/speech-emotion-recognition-/blob/main/predict_ser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip show pyaudio

In [ ]:
#import pandas as pd
import numpy as np
#import os
#import sys
#import random
# librosa is a Python library for analyzing audio and music. It can be used to extract the data from the audio files we will see it later.
import librosa
#import librosa.display
#import seaborn as sns
#import matplotlib.pyplot as plt
#sns.set_theme(style="whitegrid")
#from tqdm import tqdm
#from sklearn.preprocessing import StandardScaler, OneHotEncoder
#from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
#from sklearn.model_selection import train_test_split
# to play the audio files
#from IPython.display import Audio
#from tensorflow import keras
#from tensorflow.keras import layers
#from keras.callbacks import ReduceLROnPlateau, EarlyStopping
#from keras.models import Sequential, model_from_json
#from keras.layers import Dense, Conv1D, MaxPooling1D, Flatten, Dropout, BatchNormalization, LSTM, Bidirectional
#from keras.utils import np_utils
#from tensorflow.keras.utils import to_categorical
#from keras.callbacks import ModelCheckpoint
#import warnings
#if not sys.warnoptions:
#    warnings.simplefilter("ignore")
#warnings.filterwarnings("ignore", category=DeprecationWarning)

In [ ]:
import tensorflow as tf
#from tensorflow import keras
#from tensorflow.keras import layers
#from keras.callbacks import ReduceLROnPlateau, EarlyStopping
#from keras.models import Sequential
#from keras.layers import Dense, Conv1D, MaxPooling1D, Flatten, Dropout, BatchNormalization, LSTM, Bidirectional
#from keras.utils import np_utils, to_categorical
#from keras.callbacks import ModelCheckpoint

In [ ]:
path ='/content/drive/MyDrive/Kaggle/reel/3.wav'
path

'/content/drive/MyDrive/Kaggle/reel/3.wav'

In [ ]:
def noise(data):
    noise_amp = 0.04*np.random.uniform()*np.amax(data)
    data = data + noise_amp*np.random.normal(size=data.shape[0])
    return data


In [ ]:
#sample_rate = 22050
def extract_features(data):

    result = np.array([])

    mfccs = librosa.feature.mfcc(y=data, sr=22050, n_mfcc=58)
    mfccs_processed = np.mean(mfccs.T,axis=0)
    result = np.array(mfccs_processed)

    return result

def get_features(path):
    # duration and offset are used to take care of the no audio in start and the ending of each audio files as seen above.
    data, sample_rate = librosa.load(path, duration=3, offset=0.5)

    #without augmentation
    res1 = extract_features(data)
    result = np.array(res1)


    #noised
    noise_data = noise(data)
    res2 = extract_features(noise_data)
    result = np.vstack((result, res2))


    return result

In [ ]:
X, Y = [], []
#path = np.array(data_path.path)[0]
#for path, emotion in tqdm(zip(data_path.path, data_path.labels)):
features = get_features(path)
for feature in features:
        X.append(feature)
        # appending emotion 5 times as we have made 5 augmentation techniques on each audio file.
        # Y.append('1')

In [ ]:
print(tf.__version__)

2.12.0


In [ ]:
len(X)
X

[array([-6.70195435e+02,  6.50638504e+01,  8.88955355e-01,  1.47159796e+01,
         9.18216324e+00,  6.60576046e-01, -3.84683704e+00, -3.58394504e+00,
        -1.29590063e+01, -3.30013347e+00,  9.10780847e-01, -3.59703636e+00,
         2.37627506e+00, -4.38894129e+00,  5.45079589e-01,  8.91852617e-01,
        -4.80258989e+00, -2.10541463e+00, -1.60596776e+00, -1.05239093e+00,
        -7.06728077e+00, -6.23060465e-01, -2.72803068e+00, -5.30735970e+00,
        -1.96911705e+00, -9.46152687e-01, -5.72111034e+00,  3.32990497e-01,
        -2.54384422e+00,  1.82208031e-01, -2.35109782e+00, -2.50472760e+00,
        -3.15150547e+00, -2.19089890e+00, -3.80176091e+00, -1.81308663e+00,
        -1.26122320e+00, -2.14495444e+00, -4.15217209e+00, -1.77961588e+00,
        -4.90007544e+00, -6.85950279e-01, -3.20931625e+00, -2.98257613e+00,
        -3.46229291e+00, -2.07138038e+00, -3.75293565e+00, -2.42382929e-01,
        -3.33463597e+00, -6.66891038e-01, -1.96219957e+00, -1.45200086e+00,
        -2.2

In [ ]:
import pickle
# Charger le transformateur
with open('/content/drive/MyDrive/Kaggle/encoder.pkl', 'rb') as f:
    encoder = pickle.load(f)


"""encoder = OneHotEncoder()
Y_res = encoder.fit_transform(np.array(Y).reshape(-1,1)).toarray()
Y_res"""
encoder

OneHotEncoder()

In [ ]:
#x_train = np.expand_dims(x_train, axis=2)
X = np.expand_dims(X, axis=2)
#x_train.shape, y_train.shape, x_test.shape, y_test.shape
X.shape

(2, 58, 1)

In [ ]:
loaded_model = tf.keras.models.load_model('/content/drive/MyDrive/Kaggle/model/Emotion_Model_conv1d_gender_93.h5')

In [ ]:
preds = loaded_model.predict(X)
preds.shape

1/1 [==============================] - 0s 21ms/step


(2, 14)

In [ ]:
# predicting on test data.
y_pred = encoder.inverse_transform(preds)
#y_tmp = encoder.inverse_transform(y_test)

In [ ]:
print(y_pred)

[['female_happy']
 ['female_happy']]


In [ ]:
Lab=y_pred[0][0]
Lab

'female_happy'

In [ ]:
if Lab == 'female_angry' or Lab == 'male_angry':
    ActualLabel = 'angry'
elif Lab == 'female_disgust' or Lab == 'male_disgust':
    ActualLabel = 'disgust'
elif Lab == 'female_fear' or Lab == 'male_fear':
    ActualLabel = 'fear'
elif Lab == 'female_happy' or Lab == 'male_happy':
    ActualLabel = 'disgust'
elif Lab == 'female_sad' or Lab == 'male_sad':
    ActualLabel = 'sad'
elif Lab == 'female_surprise' or Lab == 'male_surprise':
    ActualLabel = 'surprise'
elif Lab == 'female_neutral' or Lab == 'male_neutral':
    ActualLabel = 'neutral'
else:
    ActualLabel = 'Unknown'

In [ ]:
print( ActualLabel)

neutral


In [ ]:
data_resampled = librosa.resample(data, sr_orig, sr_target)

In [ ]:
import librosa

# Record a short audio clip using your microphone and save it as a WAV file
# Replace "input.wav" with the path and filename of your WAV file
# Be sure to record a long enough clip to accurately determine the sample rate
# For example, record at least 5 seconds of audio
# You can use a tool like Audacity to record and save the audio clip
input_file = "/content/drive/MyDrive/Kaggle/reel/3.wav"

# Load the WAV file into Python using librosa
# The sample rate will be returned in the "sr" variable
data, sr = librosa.load(input_file, sr=None, mono=True)

# Print the sample rate of the loaded audio clip
print("Sample rate of microphone:", sr)

Sample rate of microphone: 22050


In [ ]:
import time
import numpy as np
import pyaudio
import tensorflow as tf
import Adafruit_CharLCD as LCD

# Load pre-trained model
model = tf.keras.models.load_model('path/to/model.h5')

# Initialize Pyaudio stream
CHUNK = 1024
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 16000
p = pyaudio.PyAudio()
stream = p.open(format=FORMAT, channels=CHANNELS, rate=RATE, input=True, frames_per_buffer=CHUNK)

# Initialize LCD screen
lcd = LCD.Adafruit_CharLCDPlate()

# Loop to stream audio and predict emotions every 3 seconds
start_time = time.time()
counter = 0
while True:
    data = np.fromstring(stream.read(CHUNK), dtype=np.int16)
    data = np.divide(data, 32768.0)  # Normalize the data
    emotion = model.predict(np.array([data]))[0]
    emotion_label = np.argmax(emotion)
    counter += 1
    if counter == int(RATE / CHUNK * 3):  # Predict emotion every 3 seconds
        if emotion_label == 0:
            lcd.clear()
            lcd.message('Emotion:\nNeutral')
        elif emotion_label == 1:
            lcd.clear()
            lcd.message('Emotion:\nCalm')
        elif emotion_label == 2:
            lcd.clear()
            lcd.message('Emotion:\nHappy')
        elif emotion_label == 3:
            lcd.clear()
            lcd.message('Emotion:\nSad')
        elif emotion_label == 4:
            lcd.clear()
            lcd.message('Emotion:\nAngry')
        elif emotion_label == 5:
            lcd.clear()
            lcd.message('Emotion:\nFearful')
        elif emotion_label == 6:
            lcd.clear()
            lcd.message('Emotion:\nDisgusted')
        elif emotion_label == 7:
            lcd.clear()
            lcd.message('Emotion:\nSurprised')
        counter = 0  # Reset counter after predicting emotion
        start_time = time.time()  # Reset start time
    if time.time() - start_time >= 3:  # Reset start time if it takes more than 3 seconds to read a chunk of audio data
        start_time = time.time()